In [ ]:
# conda activate kgforge

import getpass
import yaml
import pandas as pd
import numpy as np
import nexussdk as nxs
from kgforge.core import KnowledgeGraphForge
from kgforge.core import Resource

In [ ]:
ORGANIZATION = "fusion-adulbrich" # choose your fusion org
PROJECT = "Flower-Classification" # choose your newly created fusion project
DEPLOYMENT = "https://staging.nexus.ocp.bbp.epfl.ch/v1"

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
context = {
  "@id": "https://context.org",
  "@context": {
    "@vocab": "https://staging.nexus.ocp.bbp.epfl.ch/v1/vocabs/",
    "schema": "http://schema.org/",
    "prov": "http://www.w3.org/ns/",
    "description": {
      "@id": "http://schema.org/description"
    },
    "name": {
      "@id": "http://schema.org/name"
    },
    "used": {
      "@id": "http://www.w3.org/ns/prov#used"
    },
    "generated": {
      "@id": "http://www.w3.org/ns/prov#generated"
    }
  }
}

In [ ]:
nxs.config.set_environment(DEPLOYMENT)
nxs.config.set_token(TOKEN)
#nxs.resources.create(ORGANIZATION, PROJECT, context) #to run the first time if no context exists in your project

In [ ]:
config = {
    "Model": {
        "name": "RdfModel",
        "origin": "store",
        "source": "BlueBrainNexus",
        "context": {
            "iri": "https://context.org",
            "bucket": f"{ORGANIZATION}/{PROJECT}"
        }
    },
    "Store": {
        "name": "BlueBrainNexus",
        "endpoint": DEPLOYMENT,
        "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
        "file_resource_mapping": "https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/configurations/nexus-store/file-to-resource-mapping.hjson",
    }
}

In [ ]:
forge = KnowledgeGraphForge(config, token=TOKEN, bucket=f"{ORGANIZATION}/{PROJECT}")

### Register a Dataset and its File (distribution)

In [ ]:
myfile = forge.attach("./iris.csv")
mydataset = Resource(type="http://schema.org/Dataset", name="Iris Dataset 3" , distribution=myfile)
forge.register(mydataset)

In [ ]:
# do the same with the output

### Register an Activity

In [ ]:
myactivity = Resource(type="http://www.w3.org/ns/prov#Activity", name="testactivity8", generated=mydataset) # add generated output
myactivity

In [ ]:
forge.register(myactivity) #does not work if the resource has an array of type [..., ...]